# A coupled rainfall-runoff model in Landlab

This tutorial demonstrates a very simple synthetic rainfall-runoff model in Landlab, using the `SpatialPrecipitationDistribution` and `OverlandFlow` components. This assumes no infiltration, but it could be added by modifying the `rainfall__flux` field appropriately.

First, import the modules we'll need.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import trange

from landlab.components import OverlandFlow, SpatialPrecipitationDistribution
from landlab.io import esri_ascii

Set up a grid and load some arbitrary existing catchment elevation data. A functional version of this might use a real gauged catchment for comparison to reality.

In [ ]:
# here we use an arbitrary, very small, "real" catchment
with open("hugo_site.asc") as fp:
    grid = esri_ascii.load(fp, name="topographic__elevation", at="node")
z = grid.at_node["topographic__elevation"]

grid.status_at_node[grid.nodes_at_right_edge] = grid.BC_NODE_IS_FIXED_VALUE
grid.status_at_node[np.isclose(z, -9999.0)] = grid.BC_NODE_IS_CLOSED

grid.imshow(z, colorbar_label="Elevation (m)")

Build a mocked-up rainfall distribution using the `SpatialPrecipitationDistribution` component.

It would be trivial to replace this with an imported real rainfall field - and we save and reload the pattern to highlight how this might work.

In [ ]:
rain = SpatialPrecipitationDistribution(grid)
np.random.seed(26)  # arbitrary to get a cool-looking storm out every time

# storm lengths in hrs
for storm_t, interstorm_t in rain.yield_storms(style="monsoonal"):
    # because the rainfall comes out in mm/h
    grid.at_node["rainfall__flux"] *= 0.001

    # to make the storm heavier and more interesting!
    grid.at_node["rainfall__flux"] *= 10.0

    # plot up this storm
    grid.imshow(
        "rainfall__flux", cmap="gist_ncar", colorbar_label="Rainfall flux (m/h)"
    )
    plt.show()

Now, load the rainfall files and set up the model, telling the flood router to accept the rainfalls in the file(s) as inputs. 

In the first instance, this is set up as an instantaneous storm, with all the water dropped over the catchment in one go. Below, we modify this assumption to allow time distributed rainfall.

In [ ]:
# a veneer of water stabilises the model
grid.at_node["surface_water__depth"].fill(1.0e-12)
grid.at_node["surface_water__depth"] += grid.at_node["rainfall__flux"] * storm_t

of = OverlandFlow(grid, steep_slopes=True)

# storm_t here is the duration of the rainfall, from the rainfall component
# We're going to assume the rainfall arrives effectively instantaneously, but
# adding discharge during the run is completely viable

node_of_max_q = 2126  # established by examining the output of a previous run
outlet_depth = []
outlet_times = []

# while post_storm_elapsed_time < 0.5 * 3600.0:  # plot 30 mins-worth of runoff
dt = 1.0
for step in trange(30 * 60):
    time = step * dt

    of.run_one_step(dt=dt)
    if step % 180 == 0:
        plt.figure()
        grid.imshow("surface_water__depth", var_name="Stage (m)")
        plt.title(f"Stage at t={time} s")

    outlet_depth.append(grid.at_node["surface_water__depth"][node_of_max_q])
    outlet_times.append(time)

Now, plot the time series at the outlet (defined as the node that experiences peak stage):

In [ ]:
plt.plot(outlet_times, outlet_depth, "-")
plt.xlabel("Time elapsed (s)")
plt.ylabel("Flood stage (m)")

We can relax the assumption that all this discharge is delivered instantaneously at the start of the run with some tweaking of the driver:

In [ ]:
grid.at_node["surface_water__depth"].fill(1.0e-12)

of = OverlandFlow(grid, steep_slopes=True)

node_of_max_q = 2126
outlet_depth = []
outlet_times = []
dt = 1.0
for step in trange(60 * 60):
    time = step * dt

    of.run_one_step(dt=dt)

    if step % 600 == 0:
        plt.figure()
        grid.imshow("surface_water__depth", var_name="Stage (m)")
        plt.title(f"Stage at t={time} s")

    outlet_depth.append(grid.at_node["surface_water__depth"][node_of_max_q])
    outlet_times.append(time)

    if time < storm_t * 3600.0:
        grid.at_node["surface_water__depth"] += (
            grid.at_node["rainfall__flux"] * dt / 3600.0
        )

In [ ]:
plt.plot(outlet_times, outlet_depth, "-")
plt.xlabel("Time elapsed (s)")
plt.ylabel("Flood stage (m)")

As expected, a more realistic spread of the rainfall across the storm gives a longer and more subdued flood pulse.

(An aside: the levelling off of the tail at h~0.125m is likely due to the permanent filling of a depression in the topography - the same thing is probably causing the deep pixels in the flow maps - or are these numerical instabilities? Resolving this is left as an exercise for the reader...)